In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fiona.crs import from_epsg
from shapely.geometry import Point, Polygon
import pysal as ps
import pylab as pl
%pylab inline
import seaborn
import re

Populating the interactive namespace from numpy and matplotlib


# 1. Playground: Data Cleaning

In [2]:
pg = gpd.read_file(filename='./playground/playground.shp')
print "# of all pg = {}".format(pg.shape[0])

# Delete the data with Accessible == N
# pg = pg[pg.Accessible != 'N']
# print "# of pg with Accessible == 'Y' = {}".format(pg.shape[0])

# keep the data with Status == 'open to the public' 
# pg = pg[pg['Status'].apply(lambda x: x != None)]
# pg.Status = map(str.lower, map(str, pg.Status))
# pg = pg[pg.Status == 'open to the public']
# print "# of pg with Accessible == 'Y' & Status =='open to the public' = {}".format(pg.shape[0])

# Reset index
pg = pg.reset_index(drop =True)

# pg with selected cols
pg = pg.loc[:,['Location', 'Name', 'geometry']]

# View
print "pg.crs = {}".format(pg.crs)
pg.head(2)

# of all pg = 1236
pg.crs = {'init': u'epsg:4326'}


,Location,Name,geometry
0,"Noble, Franklin, Milton Sts",American Playground,POINT (-73.9579 40.7288)
1,BAY PKWY & CROPSEY AVENUE,Bensonhurst Park,POINT (-73.99979999999999 40.5969)


In [3]:
# clean outlier, missing lat, based on obeservation in QGIS.
pg = pg[pg.Name != 'Slide Mountain']

In [4]:
pg.to_file('pg')

<br/>
# 2. Restaurant: Data Cleaning

In [5]:
# rr for restaurant
rr = pd.read_csv('NYCRestaurant.csv', header=None)
rr = rr.iloc[:, [0,1,2]]
rr = rr.rename(columns={0: 'info', 1: 'lat', 2: 'lon'})

# geometry
rr['lonlat'] = zip(rr.lon, rr.lat)
rr['geometry'] = rr[['lonlat']].applymap(lambda x: Point(x))
rr = rr.loc[:, ['info', 'geometry']]

# to GeoDataFrame
rr = gpd.GeoDataFrame(rr)
rr.crs = pg.crs

In [6]:
rr.to_file("rr")